In [ ]:
#AURAZO ALVARADO FERNANDO MIGUEL
#ESPINOLA RONDOY ERICK ALONSO
#GIL GONZALEZ SERGIO ANDERSON
#ROBLES GONZALEZ RICHARD DANIEL

In [ ]:
#1.	Revise e implemente la interfaz basandose en el snippet de la pagina siguiente.

In [1]:
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np 
import cv2

def escalar_imagen(image, max_width):
    original_height,original_width = image.shape
    ratio = max_width / original_width
    height=int(original_height*ratio)
    return cv2.resize(image,(max_width,height))

def mostrar_imagen(image,label,max_width):
    imagen_resized=escalar_imagen(image,max_width)
    imagen_tk=ImageTk.PhotoImage(image=Image.fromarray(imagen_resized))
    label.config(image=imagen_tk)
    label.image=imagen_tk

def actualizar_filtro(valor):
    mostrar_imagen(img, label_matriz, 300)
    mostrar_imagen(img, label_g_real,300)

image_path="CONEJO.jpg"    
img=cv2.imread(image_path,0)

ventana=tk.Toplevel()
ventana.title("Mostrar Imagen y Matriz con Tkinter")

label_imagen=tk.Label(ventana)
mostrar_imagen(img,label_imagen,300)
label_imagen.grid(row=0, column=0,padx=10,pady=10)

label_matriz=tk.Label(ventana)
label_matriz.grid(row=0, column=1,padx=10,pady=10)

label_g_real=tk.Label(ventana)
label_g_real.grid(row=0,column=2,padx=10,pady=10)

slider_n = tk.Scale(ventana, from_=1,to=20,orient=tk.HORIZONTAL,
            label="Valor de n",command=actualizar_filtro)
slider_n.set(2)
slider_n.grid(row=1,column=0,columnspan=3,pady=10)

ventana.mainloop()

In [11]:
#2.	Implemente un algoritmo que le permita tener una interfaz con un slider para controlar el nivel de suavizado de una imagen. 
#El slider controlara el tamaño del filtro frecuencia que se aplicará. Puede aplicar cualquiera de los filtros pasabajos 
#mostrados. Se debe poder ver la imagen original, el filtro frecuencial aplicado y el resultado.

In [2]:
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np
import cv2
from scipy.ndimage import gaussian_filter

def escalar_imagen(image, max_width):
    original_height, original_width = image.shape
    ratio = max_width / original_width
    height = int(original_height * ratio)
    return cv2.resize(image, (max_width, height))

def aplicar_filtro_pasabajos(image, kernel_size):
    return gaussian_filter(image, sigma=kernel_size)

def generar_mascara_gaussiana(fil, col, D0, kernel_size):
    f, c = np.ogrid[0:fil, 0:col]
    D = np.sqrt((f - fil/2)**2 + (c - col/2)**2)
    H = np.exp(-(D**2) / (2 * ((D0 + kernel_size)**2)))  # Ajusta el tamaño del kernel
    return H

def mostrar_imagenes(imagenes, labels, max_width):
    for i, (image, label) in enumerate(zip(imagenes, labels)):
        if i == 1:  # Si es la máscara, normalizar y escalar
            image = (image - image.min()) / (image.max() - image.min()) * 255
        imagen_resized = escalar_imagen(image, max_width)
        imagen_tk = ImageTk.PhotoImage(image=Image.fromarray(imagen_resized))
        label.config(image=imagen_tk)
        label.image = imagen_tk

def actualizar_filtro(valor):
    kernel_size = int(valor)
    imagen_suavizada = aplicar_filtro_pasabajos(img, kernel_size)
    mascara_gaussiana = generar_mascara_gaussiana(fil, col, 0.1*fil, kernel_size)
    mostrar_imagenes([img, mascara_gaussiana, imagen_suavizada], [label_original, label_filtro, label_filtrada], 300)

image_path = "CONEJO.jpg"
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
fil, col = img.shape

ventana = tk.Toplevel()
ventana.title("Suavizado de Imagen con Filtro Pasabajos")

label_original = tk.Label(ventana)
label_filtro = tk.Label(ventana)
label_filtrada = tk.Label(ventana)

mostrar_imagenes([img, np.zeros_like(img), img], [label_original, label_filtro, label_filtrada], 300)

label_original.grid(row=0, column=0, padx=10, pady=10)
label_filtro.grid(row=0, column=1, padx=10, pady=10)
label_filtrada.grid(row=0, column=2, padx=10, pady=10)

slider_kernel_size = tk.Scale(ventana, from_=1, to=20, orient=tk.HORIZONTAL,
                              label="Tamaño del kernel", command=actualizar_filtro)
slider_kernel_size.set(2)
slider_kernel_size.grid(row=1, column=0, columnspan=3, pady=10)

ventana.mainloop()


C:\Users\DELL\anaconda3\envs\pdsei23\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide


In [9]:
#3.	 Implemente un algoritmo que le permita tener una interfaz para controlar la ganancia y tamaño de un 
#filtro de alto aumento. Se debe poder ver la imagen original, el filtro frecuencial aplicado y el 
#resultado

In [3]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import ttk
from matplotlib import pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

def apply_high_boost_filter(image_path, A, filter_size):
    # Leer la imagen
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Aplicar la Transformada de Fourier 2D
    f_transform = np.fft.fft2(img)
    fshift = np.fft.fftshift(f_transform)

    # Calcular la magnitud del espectro de frecuencia
    magnitude_spectrum = 20 * np.log(np.abs(fshift))

    # Crear un filtro pasa altos
    rows, cols = img.shape
    crow, ccol = rows // 2, cols // 2
    mask = np.ones((rows, cols), np.uint8)
    r = filter_size  # Tamaño del filtro, controlado por el slider
    center = [crow, ccol]
    x, y = np.ogrid[:rows, :cols]
    mask_area = (x - center[0]) ** 2 + (y - center[1]) ** 2 <= r*r
    mask[mask_area] = 0

    # Aplicar el filtro high-boost
    high_boost_filter = (A - 1) + mask

    # Aplicar el filtro high-boost al espectro de frecuencia
    fshift = fshift * high_boost_filter

    # Invertir la Transformada de Fourier
    f_ishift = np.fft.ifftshift(fshift)
    img_back = np.fft.ifft2(f_ishift)
    img_back_1 = np.abs(img_back)

    return img, mask, img_back_1

def update_images():
    A_value = A_slider.get()  # Obtener el valor actual de A desde el slider
    filter_size_value = filter_size_slider.get()  # Obtener el tamaño del filtro desde el slider
    img_original, mask, img_high_boost = apply_high_boost_filter(image_path, A_value, filter_size_value)

    # Mostrar las imágenes en la interfaz gráfica
    ax1.clear()
    ax1.imshow(img_original, cmap='gray')
    ax1.set_title('Imagen Original')

    ax2.clear()
    ax2.imshow(mask, cmap='gray')
    ax2.set_title('Filtro Hhp')

    ax3.clear()
    ax3.imshow(img_high_boost, cmap='gray')
    ax3.set_title(f'Imagen con Filtro High-Boost (A={A_value:.2f})')

    canvas.draw()  # Redibujar el lienzo con las imágenes actualizadas

# Crear la interfaz gráfica
root = tk.Toplevel()
root.title("High-Boost Filter")

# Ruta de la imagen
image_path = "CONEJO.jpg"  # Reemplaza con la ruta de tu imagen

# Crear slider para ajustar el valor de A
A_slider = ttk.Scale(root, from_=0, to=5, orient=tk.HORIZONTAL, length=200, command=lambda x: update_images())
A_slider.set(2.0)  # Valor inicial de A
A_slider.pack(pady=10)

# Crear slider para ajustar el tamaño del filtro
filter_size_slider = ttk.Scale(root, from_=1, to=100, orient=tk.HORIZONTAL, length=200, command=lambda x: update_images())
filter_size_slider.set(30)  # Tamaño inicial del filtro
filter_size_slider.pack(pady=5)

# Crear subplots para mostrar las imágenes
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10, 3))


# Crear el lienzo para mostrar las imágenes en la interfaz gráfica
canvas = FigureCanvasTkAgg(fig, master=root)
canvas_widget = canvas.get_tk_widget()
canvas_widget.pack()
update_images()

# Iniciar el bucle principal de la interfaz gráfica
root.mainloop()



Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\DELL\anaconda3\envs\pdsei23\lib\tkinter\__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-3-8574b62bb923>", line 70, in <lambda>
    A_slider = ttk.Scale(root, from_=0, to=5, orient=tk.HORIZONTAL, length=200, command=lambda x: update_images())
  File "<ipython-input-3-8574b62bb923>", line 44, in update_images
    filter_size_value = filter_size_slider.get()  # Obtener el tamaño del filtro desde el slider
NameError: name 'filter_size_slider' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\DELL\anaconda3\envs\pdsei23\lib\tkinter\__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-3-8574b62bb923>", line 75, in <lambda>
    filter_size_slider = ttk.Scale(root, from_=1, to=100, orient=tk.HORIZONTAL, length=200, command=lambda x: update_images())
  File "<ipython-input-3-8574b62bb923>", lin